# `mammos_entity.io`: reading and writing entities

In [25]:
from pathlib import Path

import mammos_entity as me
import mammos_units as u
from mammos_entity.io import EntityCollection

## Supported file format

`mammos_entity.io` can read and write csv and yaml files containing entity like objects (entities, quantities, or other array-like data). For entities and quantities information about ontology and units are included as additional metadata. Details of the file formats are explained in the [`mammos-entity.io` api reference](https://mammos-project.github.io/mammos/api/mammos_entity.io.html).

We create some artificial data that we can write to file:

In [26]:
Ms = me.Ms([600, 650, 700], "kA/m")
T = me.T([1, 2, 3])
theta_angle = [0, 0.5, 0.7] * u.rad
demag_factor = me.Entity("DemagnetizingFactor", [1 / 3, 1 / 3, 1 / 3])
comments = ["Some comment", "Some other comment", "A third comment"]

## MaMMoS CSV

### Writing
We can write data to a csv file as shown in the following cell. Names of the keyword arguments determine column names in the file.

In [27]:
me.io.entities_to_file(
    "example.csv",
    "Example csv file.\nThe description can have multiple lines.\n\nLines can also be empty.",
    Ms=Ms,
    T=T,
    angle=theta_angle,
    demag_factor=demag_factor,
    comment=comments,
)

This has produced the following file:

In [28]:
print(Path("example.csv").read_text())

#mammos csv v2
#----------------------------------------
# Example csv file.
# The description can have multiple lines.
# 
# Lines can also be empty.
#----------------------------------------
#SpontaneousMagnetization,ThermodynamicTemperature,,DemagnetizingFactor,
#https://w3id.org/emmo/domain/magnetic_material#EMMO_032731f8-874d-5efb-9c9d-6dafaa17ef25,https://w3id.org/emmo#EMMO_affe07e4_e9bc_4852_86c6_69e26182a17f,,https://w3id.org/emmo/domain/magnetic_material#EMMO_0f2b5cc9-d00a-5030-8448-99ba6b7dfd1e,
#kA / m,K,rad,,
Ms,T,angle,demag_factor,comment
600.0,1.0,0.0,0.3333333333333333,Some comment
650.0,2.0,0.5,0.3333333333333333,Some other comment
700.0,3.0,0.7,0.3333333333333333,A third comment



### Reading
We can read it back in and get a container object (called EntityCollection) containing all columns:

In [29]:
content = me.io.entities_from_file("example.csv")
content

EntityCollection(
    Ms=Entity(ontology_label='SpontaneousMagnetization', value=array([600., 650., 700.]), unit='kA / m'),
    T=Entity(ontology_label='ThermodynamicTemperature', value=array([1., 2., 3.]), unit='K'),
    angle=<Quantity [0. , 0.5, 0.7] rad>,
    demag_factor=Entity(ontology_label='DemagnetizingFactor', value=array([0.33333333, 0.33333333, 0.33333333])),
    comment=array(['Some comment', 'Some other comment', 'A third comment'],
      dtype=object),
)

The recommended way of accessing the data is by using the individual elements. This preserves the correct data type:

In [30]:
content.Ms

Entity(ontology_label='SpontaneousMagnetization', value=array([600., 650., 700.]), unit='kA / m')

In [31]:
content.T

Entity(ontology_label='ThermodynamicTemperature', value=array([1., 2., 3.]), unit='K')

In [32]:
content.angle

<Quantity [0. , 0.5, 0.7] rad>

In [33]:
content.demag_factor

Entity(ontology_label='DemagnetizingFactor', value=array([0.33333333, 0.33333333, 0.33333333]))

In [34]:
content.comment

array(['Some comment', 'Some other comment', 'A third comment'],
      dtype=object)

We can also get a `pandas` dataframe of the data we have read. This is designed as a convenience functions but due to limitation of `pandas` we loose ontology information. This is why we recommend using the individual elements directly where possible. The columns names consist of short name and units (where columns have a unit):

In [35]:
content.to_dataframe()

,Ms (kA / m),T (K),angle (rad),demag_factor,comment
0,600.0,1.0,0.0,0.333333,Some comment
1,650.0,2.0,0.5,0.333333,Some other comment
2,700.0,3.0,0.7,0.333333,A third comment


We can also get a dataframe without units in the column names:

In [36]:
content.to_dataframe(include_units=False)

,Ms,T,angle,demag_factor,comment
0,600.0,1.0,0.0,0.333333,Some comment
1,650.0,2.0,0.5,0.333333,Some other comment
2,700.0,3.0,0.7,0.333333,A third comment


### Reading with `pandas`
If we only need the numerical data but not the entity information, we can also read the csv file with pandas:

In [37]:
import pandas as pd

pd.read_csv("example.csv", comment="#")

,Ms,T,angle,demag_factor,comment
0,600.0,1.0,0.0,0.333333,Some comment
1,650.0,2.0,0.5,0.333333,Some other comment
2,700.0,3.0,0.7,0.333333,A third comment


### Create, modify and save an `EntityCollection`

Manipulating an `EntityCollection` directly is not possible. Entities can however be overwritten or added to an existing collection.

In [38]:
Ms_kAm = content.Ms
Ms_Am = me.Ms(Ms_kAm, unit="A/m")
content.Ms = Ms_Am
content.A = me.A([6e-12, 7e-12, 8e-12])
content

EntityCollection(
    Ms=Entity(ontology_label='SpontaneousMagnetization', value=array([600000., 650000., 700000.]), unit='A / m'),
    T=Entity(ontology_label='ThermodynamicTemperature', value=array([1., 2., 3.]), unit='K'),
    angle=<Quantity [0. , 0.5, 0.7] rad>,
    demag_factor=Entity(ontology_label='DemagnetizingFactor', value=array([0.33333333, 0.33333333, 0.33333333])),
    comment=array(['Some comment', 'Some other comment', 'A third comment'],
      dtype=object),
    A=Entity(ontology_label='ExchangeStiffnessConstant', value=array([6.e-12, 7.e-12, 8.e-12]), unit='J / m'),
)

It is also possible to initialize an empty `EntityCollection` and to add entities as its attributes.

In [39]:
short_collection = EntityCollection()
short_collection.Ms = me.Ms(1.5e6)
short_collection.A = me.A(8e-12)
short_collection

EntityCollection(
    Ms=Entity(ontology_label='SpontaneousMagnetization', value=np.float64(1500000.0), unit='A / m'),
    A=Entity(ontology_label='ExchangeStiffnessConstant', value=np.float64(8e-12), unit='J / m'),
)

To write an `EntityCollection` to disk use the following command.

In [40]:
me.io.entities_to_file(
    "modified_example.csv", "Modified version of `example.csv`.\nMs has been converted to A/m.", **content.__dict__
)
pd.read_csv("modified_example.csv", comment="#")

,Ms,T,angle,demag_factor,comment,A
0,600000.0,1.0,0.0,0.333333,Some comment,6.000000e-12
1,650000.0,2.0,0.5,0.333333,Some other comment,7.000000e-12
2,700000.0,3.0,0.7,0.333333,A third comment,8.000000e-12


## MaMMoS YAML

### Writing
We can write data to a yaml file (extension .yaml or .yml) as shown in the following cell. Names of the keyword arguments determine keys under `data:` in the file.

In [41]:
me.io.entities_to_file(
    "example.yaml",
    "Example yaml file.\nThe description can have multiple lines.\n\nLines can also be empty.",
    Ms=Ms,
    T=T,
    angle=theta_angle,
    demag_factor=demag_factor,
    comment=comments,
)

This has produced the following file:

In [42]:
print(Path("example.yaml").read_text())

metadata:
  version: v1
  description: |-
    Example yaml file.
    The description can have multiple lines.

    Lines can also be empty.
data:
  Ms:
    ontology_label: SpontaneousMagnetization
    ontology_iri: https://w3id.org/emmo/domain/magnetic_material#EMMO_032731f8-874d-5efb-9c9d-6dafaa17ef25
    unit: kA / m
    value: [600.0, 650.0, 700.0]
  T:
    ontology_label: ThermodynamicTemperature
    ontology_iri: https://w3id.org/emmo#EMMO_affe07e4_e9bc_4852_86c6_69e26182a17f
    unit: K
    value: [1.0, 2.0, 3.0]
  angle:
    ontology_label: null
    ontology_iri: null
    unit: rad
    value: [0.0, 0.5, 0.7]
  demag_factor:
    ontology_label: DemagnetizingFactor
    ontology_iri: https://w3id.org/emmo/domain/magnetic_material#EMMO_0f2b5cc9-d00a-5030-8448-99ba6b7dfd1e
    unit: ''
    value: [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
  comment:
    ontology_label: null
    ontology_iri: null
    unit: null
    value: [Some comment, Some other comment, A third c

### Reading
We can read it back in and get a container object (called EntityCollection) containing all columns:

In [43]:
content = me.io.entities_from_file("example.yaml")
content

EntityCollection(
    Ms=Entity(ontology_label='SpontaneousMagnetization', value=array([600., 650., 700.]), unit='kA / m'),
    T=Entity(ontology_label='ThermodynamicTemperature', value=array([1., 2., 3.]), unit='K'),
    angle=<Quantity [0. , 0.5, 0.7] rad>,
    demag_factor=Entity(ontology_label='DemagnetizingFactor', value=array([0.33333333, 0.33333333, 0.33333333])),
    comment=['Some comment', 'Some other comment', 'A third comment'],
)

We can access elements as show for the csv file before, e.g.

In [44]:
content.to_dataframe()

,Ms (kA / m),T (K),angle (rad),demag_factor,comment
0,600.0,1.0,0.0,0.333333,Some comment
1,650.0,2.0,0.5,0.333333,Some other comment
2,700.0,3.0,0.7,0.333333,A third comment


### Entities of different shape and length

Unlike csv the yaml format supports entities with different lengths and shape.

In [45]:
me.io.entities_to_file("example2.yml", T=me.T([10, 20, 30]), Tc=me.Tc(300), Ms=me.Ms([[1, 2], [3, 4]]))

In [46]:
print(Path("example2.yml").read_text())

metadata:
  version: v1
  description: null
data:
  T:
    ontology_label: ThermodynamicTemperature
    ontology_iri: https://w3id.org/emmo#EMMO_affe07e4_e9bc_4852_86c6_69e26182a17f
    unit: K
    value: [10.0, 20.0, 30.0]
  Tc:
    ontology_label: CurieTemperature
    ontology_iri: https://w3id.org/emmo#EMMO_6b5af5a8_a2d8_4353_a1d6_54c9f778343d
    unit: K
    value: 300.0
  Ms:
    ontology_label: SpontaneousMagnetization
    ontology_iri: https://w3id.org/emmo/domain/magnetic_material#EMMO_032731f8-874d-5efb-9c9d-6dafaa17ef25
    unit: A / m
    value: [[1.0, 2.0], [3.0, 4.0]]



In [47]:
content = me.io.entities_from_file("example2.yml")
content

EntityCollection(
    T=Entity(ontology_label='ThermodynamicTemperature', value=array([10., 20., 30.]), unit='K'),
    Tc=Entity(ontology_label='CurieTemperature', value=np.float64(300.0), unit='K'),
    Ms=Entity(ontology_label='SpontaneousMagnetization', value=array([[1., 2.],
       [3., 4.]]), unit='A / m'),
)

In [48]:
content.to_dataframe()

ValueError: Per-column arrays must each be 1-dimensional

We can acces individual elements of the returned object as before.

In [ ]:
content.T

In [ ]:
content.Tc

In [ ]:
content.Ms

Conversion to a dataframe is however generally not possible as the shapes are incompatible with a `pandas.DataFrame`.

## Converting unformatted files and tables to `mammos csv`

Users may wish to update other files to the `mammos csv` format in order to make use of the additional functionality. 
Details of the file format are explained in the [`mammos-entity.io` api reference](https://mammos-project.github.io/mammos/api/mammos_entity.io.html).

Converting your “raw” data into this format involves three main steps:

1. Load your file into python (e.g. with `pandas`).

2. Create an `Entity`, `quanity`, or similar out of each column (by assigning the correct ontology term and/or units).

3. Export the result with `entities_to_csv`.


First let's create a file so we can see an example of how to do the conversion. We will create the following structure:
```dat
1 10.0 1.6083568305976572 -16778187.088808443
1 9.0 1.6083393931987826 -15498304.121589921
...
```

This file is quite basic, in particular, there no headers, no units, no ontology information, and the use of the space as separator rather than a comma.

Only the user knows what each of the columns are.
In this example first column is the configuration type, the second column is the value of $\mu_0 H_{\mathsf{ext}}$ in Tesla, the third column is the magnetic polarisation in Tesla and the last column is the energy density in J/m$^3$.

In [ ]:
Path("example.dat").write_text("""\
1 10.0 1.6083568305976572 -16778187.088808443
1 9.0 1.6083393931987826 -15498304.121589921
1 8.0 1.6083184361075116 -14218436.37373519
1 7.0 1.608292941666901 -12938587.029585946
1 6.0 1.6082614950059932 -11658760.230932372
""")

We can use `pandas` to read the file into python:

In [ ]:
df = pd.read_csv("example.dat", sep=" ", names=["configuration_type", "mu0_Hext", "Js", "energy_density"])
df

To rewrite this in the `mammos csv` format, we then need to associate each column with an entity, quantity, or another python object. Now is also time to do any data manipulation (such as changing units).

In this example we:
- Convert configuration type to a `numpy` array.
- Convert magnetic flux density ($\mu_0 H_{\mathsf{ext}}$) to the external magnetic field Entity using `mammos_units` for the relevant conversions.
- Convert magnetic polarisation to the corresponding entity.
- Convert energy density to the corresponding entity.

In [ ]:
configuration_type = df["configuration_type"].to_numpy()
H = me.Entity(
    ontology_label="ExternalMagneticField",
    value=(df["mu0_Hext"].to_numpy() * u.T).to(u.A / u.m, equivalencies=u.magnetic_flux_field()),
    unit=u.A / u.m,
)
Js = me.Entity(
    ontology_label="MagneticPolarisation",
    value=df["Js"],
    unit=u.T,
)
energy_density = me.Entity(ontology_label="EnergyDensity", value=df["energy_density"], unit=u.J / u.m**3)

We can now write the `mammos csv`:

In [ ]:
me.io.entities_to_file(
    "example.csv",
    "This file contains...",
    configuration_type=configuration_type,
    H=H,
    Js=Js,
    energy_density=energy_density,
)

Looking at the file produced we can see the data is now in the correct format with the ontology information included:

In [ ]:
print(Path("example.csv").read_text())

## Merging two `EntityCollection`

Reading a `mammos_entity` `csv` or `yaml` file will create an `EntityCollection` object as shown in the section above. It is possible to merge two of them using the `mammos_entity.merge` function.

In [ ]:
me.io.entities_to_file(
    "A.csv",
    "This file contains computed coercivites",
    x_pos=me.Entity("Length", [-10.0, -10.0, -10.0, -10.0, -10.0], "mm"),
    y_pos=me.Entity("Length", [-10.0, -15.0, -20.0, -25.0, -30.0], "mm"),
    a=me.Entity("LocalLatticeConstantA", [8.7838, 8.7835, 8.7834, 8.7828, 8.7824], "Angstrom"),
    c=me.Entity("LocalLatticeConstantC", [12.1634, 12.1629, 12.1626, 12.1614, 12.1611], "Angstrom"),
    volume=me.Entity("CellVolume", [938.4688596, 938.3661812, 938.3216703, 938.1009148, 938.056412], "Angstrom^3"),
    Ms=me.Ms([1205044.626, 1203469.633, 1202605.731, 1198792.8, 1197854.062]),
    A=me.A([7.071954208, 7.053474008, 7.043351497, 6.998792402, 6.987851124], "pJ/m"),
    K1=me.Ku([3.411298313, 3.388623141, 3.37627427, 3.3225042, 3.309445639], "MJ/m3"),
    Ha=me.Entity(
        "AnisotropyField",
        [4505434.713046, 4481343.84430899, 4468220.34398305, 4411045.5691224, 4397151.94905576],
        "A/m",
    ),
)

In [ ]:
me.io.entities_to_file(
    "B.csv",
    "Integral of Absolute Difference",
    x_pos=me.Entity("Length", [-1, -1, -1, -1, -1], "cm"),
    y_pos=me.Entity("Length", [-10.0, -15.0, -20.0, -25.0, -30.0], "mm"),
    integral_abs_diff=[1.176182592576099, 1.174635419947949, 1.1535128973539177, 1.204475631146945, 1.1041682311364889],
)

In [ ]:
A_collection = me.io.entities_from_file("A.csv")
A_collection

In [ ]:
B_collection = me.io.entities_from_file("B.csv")
B_collection

In [ ]:
merged_collection = me.merge(A_collection, B_collection, how="inner")
merged_collection

In [ ]:
merged_collection.to_dataframe()

It is important to note that:
- The `merge` function is a thin wrapper around the `pandas.merge` function and the merge behavior depends on the additional keyword arguments passed to `merge` (which are forwarded to `pandas.merge` internally) on top of two `EntityCollection` to be merged. Please refer to the [panda.merge](https://pandas.pydata.org/docs/reference/api/pandas.merge.html) documentation.
- If both collections contain entities with the same ontology label but different units, the `right` entity is automatically converted to the unit of the `left` entity before merging.

### Merge inner

In [ ]:
ec_1 = me.io.EntityCollection(
    x=[1, 2, 3, 3],
    y=[1, 1, 3, 3],
    Ms=me.Ms([1, 2, 3, 3.5]),
)
ec_2 = me.io.EntityCollection(
    x=[2, 3, 4, 3],
    y=[1, 3, 5, 7],
    A=me.A([22, 33, 44, 33.55]),
)
ec_merged_1 = me.merge(ec_1, ec_2, on="x", how="inner")
ec_merged_1

In [ ]:
ec_merged_2 = me.merge(ec_1, ec_2, on=["x", "y"], how="inner")
ec_merged_2

### Merge product with overlap

In [ ]:
ec_1 = me.io.EntityCollection(x=[1, 2, 3], Ms=me.Ms([1, 2, 3]))
ec_2 = me.io.EntityCollection(
    x=[2, 3, 4],
    Ms=me.Ms([22, 33, 44]),
)
ec_merged_1 = me.merge(ec_1, ec_2, on="x")
ec_merged_1

In [ ]:
ec_merged_2 = me.merge(ec_1, ec_2, on="x", suffixes=("_1", "_2"))
ec_merged_2

### Merge inner different units

In [ ]:
ec_1 = me.io.EntityCollection(
    x=[1, 2, 3] * u.m,
    Ms=me.Ms([1, 2, 3]),
)
ec_2 = me.io.EntityCollection(
    x=[2000, 3000, 4000] * u.mm,
    Ms=me.Ms([22, 33, 44]),
)
ec_merged_1 = me.merge(ec_1, ec_2, on="x")
ec_merged_1

In [ ]:
ec_merged_2 = me.merge(ec_2, ec_1, on="x")
ec_merged_2

### Merge left

In [ ]:
ec_1 = me.io.EntityCollection(
    x=[1, 2, 3],
    y=[1, 2, 3],
    Ms=me.Ms([1, 2, 3]),
)
ec_2 = me.io.EntityCollection(
    x=[2, 3, 4],
    y=[2, 3, 4],
    A=me.A([2, 3, 4]),
)
ec_merged_left = me.merge(ec_1, ec_2, on=["x", "y"], how="left")
ec_merged_left

### Merge right

In [ ]:
ec_1 = me.io.EntityCollection(
    x=[1, 2, 3],
    y=[1, 2, 3],
    Ms=me.Ms([1, 2, 3]),
)
ec_2 = me.io.EntityCollection(
    x=[2, 3, 4],
    y=[2, 3, 4],
    A=me.A([2, 3, 4]),
)
ec_merged_right = me.merge(ec_1, ec_2, on=["x", "y"], how="right")
ec_merged_right

### Merge outer

In [ ]:
ec_1 = me.io.EntityCollection(x=[1, 2, 3], y=[1, 2, 3], Ms=me.Ms([1, 2, 3]))
ec_2 = me.io.EntityCollection(
    x=[2, 3, 4],
    y=[2, 3, 4],
    Ms=me.Ms([2, 3, 4]),
)
ec_merged_outer = me.merge(ec_1, ec_2, on=["x", "y"], how="outer")
ec_merged_outer

### Merge cross

In [49]:
ec_1 = me.io.EntityCollection(
    x=[1, 2, 3],
    # y=[1, 2, 3],
    Ms=me.Ms([1, 2, 3]),
)
ec_2 = me.io.EntityCollection(
    # x=[2, 3, 4],
    y=[2, 3, 4],
    Ms_other=me.Ms([2, 3, 4]),
)
ec_merged_cross = me.merge(ec_1, ec_2, how="cross")
ec_merged_cross

EntityCollection(
    x=array([1, 1, 1, 2, 2, 2, 3, 3, 3]),
    Ms=Entity(ontology_label='SpontaneousMagnetization', value=array([1., 1., 1., 2., 2., 2., 3., 3., 3.]), unit='A / m'),
    y=array([2, 3, 4, 2, 3, 4, 2, 3, 4]),
    Ms_other=Entity(ontology_label='SpontaneousMagnetization', value=array([2., 3., 4., 2., 3., 4., 2., 3., 4.]), unit='A / m'),
)

### Merge different names

In [48]:
ec_1 = me.io.EntityCollection(
    x_array=[1, 2],
    y_array=[1, 2],
    Ms=me.Ms([100, 200]),
)
ec_2 = me.io.EntityCollection(
    x=[1, 2],
    y=[1, 2],
    A=me.A([0.8, 0.8]),
)
ec_merged_1 = me.merge(ec_1, ec_2, left_on=["x_array", "y_array"], right_on=["x", "y"], how="inner")
ec_merged_1

EntityCollection(
    x_array=array([1, 2]),
    y_array=array([1, 2]),
    Ms=Entity(ontology_label='SpontaneousMagnetization', value=array([100., 200.]), unit='A / m'),
    x=array([1, 2]),
    y=array([1, 2]),
    A=Entity(ontology_label='ExchangeStiffnessConstant', value=array([0.8, 0.8]), unit='J / m'),
)

### Merge Indicator

In [50]:
ec_1 = me.io.EntityCollection(
    x=[1, 2, 3],
    y=[1, 2, 3],
    Ms=me.Ms([1, 2, 3]),
)
ec_2 = me.io.EntityCollection(
    x=[2, 3, 4],
    y=[2, 3, 4],
    Ms=me.Ms([2, 3, 4]),
)
ec_merged = me.merge(ec_1, ec_2, on=["x", "y"], how="outer", indicator=True)
ec_merged

EntityCollection(
    x=array([1, 2, 3, 4]),
    y=array([1, 2, 3, 4]),
    Ms_x=Entity(ontology_label='SpontaneousMagnetization', value=array([ 1.,  2.,  3., nan]), unit='A / m'),
    Ms_y=Entity(ontology_label='SpontaneousMagnetization', value=array([nan,  2.,  3.,  4.]), unit='A / m'),
    _merge=0     left_only
1          both
2          both
3    right_only
Name: _merge, dtype: category
Categories (3, object): ['left_only', 'right_only', 'both'],
)

In [51]:
ec_merged.to_dataframe()

,x,y,Ms_x (A / m),Ms_y (A / m),_merge
0,1,1,1.0,NaN,left_only
1,2,2,2.0,2.0,both
2,3,3,3.0,3.0,both
3,4,4,NaN,4.0,right_only


In [50]:
ec_merged._merge

0     left_only
1          both
2          both
3    right_only
Name: _merge, dtype: category
Categories (3, object): ['left_only', 'right_only', 'both']

In [51]:
type(ec_merged._merge)

pandas.core.series.Series

>**NOTE**: The value of `_merge` looks wrong. We should fix this or not support indicator=True argument.

### Merge different entities same name

In [52]:
ec_1 = me.io.EntityCollection(
    Ms=me.Ms([1, 2, 3, 4]),
    x=[10, 20, 30, 40],
)
ec_2 = me.io.EntityCollection(
    Ms=me.B([1, 2, 3, 4], "T"),
    x=[10, 20, 30, 40],
)
me.merge(ec_1, ec_2)

ValueError: incompatible ontology labels for 'SpontaneousMagnetization(value=[1. 2. 3. 4.], unit=A / m)' and 'MagneticFluxDensity(value=[1. 2. 3. 4.], unit=T)'

### Merge different units same name

In [53]:
ec_1 = me.io.EntityCollection(
    Ms=me.Ms([1, 2, 3, 4]),
    x=[10, 20, 30, 40],
)
ec_2 = me.io.EntityCollection(
    Ms=[1, 2, 3, 4] * u.T,
    x=[10, 20, 30, 40],
)
me.merge(ec_1, ec_2)

ValueError: incompatible units for 'SpontaneousMagnetization(value=[1. 2. 3. 4.], unit=A / m)' and '[1. 2. 3. 4.] T'

In [54]:
me.merge(ec_1, ec_2, how="right")

ValueError: incompatible units for '[1. 2. 3. 4.] T' and 'SpontaneousMagnetization(value=[1. 2. 3. 4.], unit=A / m)'

In [55]:
ec_1.Ms = [1, 2, 3, 4] * u.A / u.m

In [56]:
me.merge(ec_1, ec_2)

ValueError: incompatible units for '[1. 2. 3. 4.] A / m' and '[1. 2. 3. 4.] T'

In [57]:
me.merge(ec_1, ec_2, how="right")

ValueError: incompatible units for '[1. 2. 3. 4.] T' and '[1. 2. 3. 4.] A / m'

### Merge how

In [60]:
ec_1 = me.io.EntityCollection(
    Ms=me.Ms([1, 2, 3]),
    x=[10, 20, 30],
)
ec_2 = me.io.EntityCollection(
    Ms=[2, 3, 4] * u.A / u.m,
    y=[60, 70, 80],
)

merge_left = me.merge(ec_1, ec_2)
merge_right = me.merge(ec_1, ec_2, left_on="Ms", right_index=True)

In [61]:
merge_left

EntityCollection(
    Ms=Entity(ontology_label='SpontaneousMagnetization', value=array([2., 3.]), unit='A / m'),
    x=array([20, 30]),
    y=array([60, 70]),
)

In [62]:
merge_right

EntityCollection(
    Ms=Entity(ontology_label='SpontaneousMagnetization', value=array([1., 2.]), unit='A / m'),
    Ms_x=Entity(ontology_label='SpontaneousMagnetization', value=array([1., 2.]), unit='A / m'),
    x=array([10, 20]),
    Ms_y=Entity(ontology_label='SpontaneousMagnetization', value=array([3., 4.]), unit='A / m'),
    y=array([70, 80]),
)

In [55]:
ec_1.Ms = [1000, 2000, 3000, 4000] * u.mA / u.m
merge_left = me.merge(ec_1, ec_2)
merge_right = me.merge(ec_1, ec_2, how="right")

ValueError: All arrays must be of the same length

In [62]:
merge_left

EntityCollection(
    Ms=<Quantity [1000., 2000., 3000., 4000.] mA / m>,
    x=array([10, 20, 30, 40]),
    y=array([50, 60, 70, 80]),
)

In [63]:
merge_right

EntityCollection(
    Ms=<Quantity [1., 2., 3., 4.] A / m>,
    x=array([10, 20, 30, 40]),
    y=array([50, 60, 70, 80]),
)

### Merge `left_on` `right_on` 

In [64]:
ec1 = me.io.EntityCollection(
    T_K=me.T([1, 2, 3], "K"),
    Ms=me.Ms([40, 50, 60]),
)
ec2 = me.io.EntityCollection(
    T_mK=me.T([1, 2, 3], "mK"),
    A=me.A([70, 80, 90]),
)
ec_merged = me.merge(ec1, ec2, left_on="T_K", right_on="T_mK")
ec_merged

EntityCollection(
    T_K=Entity(ontology_label='ThermodynamicTemperature', value=array([], dtype=float64), unit='K'),
    Ms=Entity(ontology_label='SpontaneousMagnetization', value=array([], dtype=float64), unit='A / m'),
    T_mK=Entity(ontology_label='ThermodynamicTemperature', value=array([], dtype=float64), unit='K'),
    A=Entity(ontology_label='ExchangeStiffnessConstant', value=array([], dtype=float64), unit='J / m'),
)

### Merge on

In [66]:
ec1 = me.io.EntityCollection(
    x=[1, 2, 3],
    magnetisation=me.Ms([100, 200, 300]),
)
ec2 = me.io.EntityCollection(
    x=[1, 2, 3],
    magnetisation=me.Js([1, 2, 3]),
)
merged_collection = me.merge(ec1, ec2, on="x", suffixes=["_sim", "_exp"])
merged_collection

EntityCollection(
    x=array([1, 2, 3]),
    magnetisation_sim=Entity(ontology_label='SpontaneousMagnetization', value=array([100., 200., 300.]), unit='A / m'),
    magnetisation_exp=Entity(ontology_label='SpontaneousMagneticPolarisation', value=array([1., 2., 3.]), unit='V s / m2'),
)